In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import Path
import shutil
import numpy as np
import thuner.data as data
import thuner.data.dispatch as dispatch
import thuner.grid as grid
import thuner.track as track
import thuner.option as option
import thuner.visualize as visualize
import thuner.analyze as analyze
import thuner.option as option
import thuner.attribute as attribute
import thuner.utils as utils

notebook_name = "cpol_demo.ipynb"

In [ ]:
# Parent directory for saving outputs
base_local = Path.home() / "THUNER_output"
start = "2005-11-13T18:00:00"
end = "2005-11-13T20:00:00"

output_directory = base_local / "runs/cpol/cpol_demo_geographic"

if output_directory.exists():
    shutil.rmtree(output_directory)

output_directory.mkdir(parents=True, exist_ok=True)
options_directory = output_directory / "options"
options_directory.mkdir(parents=True, exist_ok=True)

times_dict = {"start": start, "end": end}
cpol_options = data.data.CPOLOptions(**times_dict)
era5_dict = {"latitude_range": [-14, -10], "longitude_range": [129, 133]}
era5_pl_options = data.data.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.data.ERA5Options(**times_dict, **era5_dict)
data_options = data.data.DataOptions(
    datasets=[cpol_options, era5_pl_options, era5_sl_options]
)
data_options.to_yaml(options_directory / "data.yml")

# altitude = list(np.arange(0, 20e3 + 500, 500))
# altitude = [float(alt) for alt in altitude]
# grid_options = grid.create_options(name="geographic", altitude=altitude)
# grid.check_options(grid_options)
# grid.save_grid_options(grid_options, options_directory)

grid_options = grid.GridOptions()
grid_options.to_yaml(options_directory / "grid.yml")

# Create the track_options dictionary
track_options = track.default_track_options(dataset="cpol")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options["middle_echo"] = visualize.option.runtime_options(
    "middle_echo", save=True, style="presentation", figure_types=["mask"]
)
# visualize.option.save_display_options(visualize_options, options_directory)
visualize_options = None

In [ ]:
times = data.utils.generate_times(data_options.dataset_by_name("cpol"))
tracks = track.track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory
)

# Cartesian Coordinates

In [ ]:
output_directory = base_local / "runs/cpol/cpol_demo_cartesian"
options_directory = output_directory / "options"
options_directory.mkdir(parents=True, exist_ok=True)

if output_directory.exists():
    shutil.rmtree(output_directory)

# grid_options = grid.create_options(name="cartesian", regrid=False, altitude=altitude)
# grid.check_options(grid_options)
# grid.save_grid_options(grid_options, options_directory)

grid_options = grid.GridOptions(name="cartesian", regrid=False)
grid_options.to_yaml(options_directory / "grid.yml")
data_options.to_yaml(options_directory / "data.yml")
track_options.to_yaml(options_directory / "track.yml")

# visualize.option.save_display_options(visualize_options, options_directory)

times = data.utils.generate_times(data_options.dataset_by_name("cpol"))
tracks = track.track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

# Analysis

In [ ]:
analysis_options = analyze.mcs.AnalysisOptions()
analysis_options.to_yaml(options_directory / "analysis.yml")
# utils.save_options(analysis_options, filename="analysis", options_directory=output_directory / "options")
analyze.mcs.process_velocities(output_directory)
analyze.mcs.quality_control(output_directory, analysis_options)
analyze.mcs.classify_all(output_directory, analysis_options)

In [ ]:
figure_options = visualize.option.horizontal_attribute_options(
    "cpol_20051113", style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64("2005-11-13T18:00")
end_time = np.datetime64("2005-11-13T19:50")
visualize.attribute.mcs_series(output_directory, start_time, end_time, figure_options, parallel_figure=True, by_date=False)